# 1. Candidate Word Generation
## Objective: 
Identify candidate words for substitution or insertion that do not drastically change the sentence's meaning.
## Implementation:
Uses a masked language model (e.g., BERT) to predict possible replacements or insertions for each word.

### For substitution, 
each word in the sentence is masked one at a time, and BERT predicts the top-k tokens for that position.
### For insertion, 
additional tokens are proposed by masking at various positions between existing words.

## Code Component:

In [ ]:
from textattack.transformations import WordSwapMaskedLM
transformation = WordSwapMaskedLM(method="bae", max_candidates=50)

# This uses BERT's predictions to generate up to 50 candidate replacements or insertions.

# 2. Scoring and Ranking
## Objective: 
Rank the candidate words by their potential to cause a model misclassification while maintaining semantic similarity.
## Implementation:
Each candidate is scored using the model's confidence on the adversarial target.
Candidates are also checked for semantic similarity using constraints like cosine similarity in embedding space.
## Code Component:

In [ ]:
from textattack.constraints.semantics import WordEmbeddingDistance
constraints = [WordEmbeddingDistance(min_cos_sim=0.8)]
# This ensures that candidates are semantically close to the original word.

# 3. Word Substitution or Insertion
## Objective: 
Modify the input sentence based on the ranked list of candidates.
## Implementation:
For substitution, the highest-ranked replacement is applied to the original word.

For insertion, the highest-ranked token is inserted at the identified position.

The process continues iteratively until the attack succeeds or a limit on changes is reached.
## Code Component: 
The transformation (e.g., substitution or insertion) is applied iteratively:

In [ ]:
from textattack.constraints import MaxWordsPerturbed
max_changes = MaxWordsPerturbed(max_percent=0.2)  # Limit to 20% changes

# 4. Adversarial Verification
## Objective: 
Verify that the modified sentence successfully fools the target model.
## Implementation:
The adversarial example is tested against the model to confirm misclassification.

If the attack is successful, the example is returned as an adversarial instance.

If not, the process continues with the next ranked candidate or phase.
## Code Component: 
This is handled by the attack's goal function, which specifies the conditions for success:

In [ ]:
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model)

# Integration in TextAttack

In TextAttack, these phases are combined and executed automatically in the BAEGarg2019 recipe:

In [ ]:
from textattack.attack_recipes import BAEGarg2019
from textattack.models.wrappers import HuggingFaceModelWrapper

# Load a model and build the BAE attack recipe
model = HuggingFaceModelWrapper.from_pretrained("bert-base-uncased")
attack = BAEGarg2019.build(model)

# Attack sample sentences
results = attack.attack_dataset([("This is a sample sentence.", 0)])